In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE

In [3]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
Market.open("BN", "BTCBUSD")

In [5]:
rbot.Market.download(10)


file to download: 'BTCBUSD-trades-2022-12-06.zip'
will be located under: '"/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/.tmpnwbSv0/BTCBUSD-trades-2022-12-06.zip"'
file to download: 'BTCBUSD-trades-2022-12-07.zip'
will be located under: '"/var/folders/5r/xmxmz7gj2cjcb7d65tgwvrt40000gn/T/.tmp0hanLk/BTCBUSD-trades-2022-12-07.zip"'


In [6]:
rbot.Market.cache_data()

# rbot.init_debug_log()




In [9]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [10]:


print("start")
r = back_tester.run(Agent(), rbot.DAYS_BEFORE(10), 0)
#r = back_tester.run(Agent(), 0, 0)
print("end")




start
end


In [11]:
import rbot
df = rbot.result_to_df(r)

In [12]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-11-28 15:59:59.766000+00:00,2022-11-28 16:00:03.854000+00:00,0000-0001,0,Sell,True,Open,16140.90,0.01,161.4090,0.00,...,16140.90,0.01,161.4090,0.0000,0.161409,-0.161409,-0.01,Open Short,-0.161409,-0.01
2022-11-28 17:59:59.790000+00:00,2022-11-28 18:00:00.443000+00:00,0000-0002,0,Buy,True,Close,16140.90,0.01,161.4090,16201.60,...,16201.60,0.01,162.0160,-0.6070,0.162016,-0.769016,0.01,doten Long,-0.930425,0.00
2022-11-28 17:59:59.790000+00:00,2022-11-28 18:00:00.443000+00:00,0000-0002,1,Buy,True,Open,16201.60,0.01,162.0160,0.00,...,16201.60,0.01,162.0160,0.0000,0.162016,-0.162016,0.01,doten Long,-1.092441,0.01
2022-11-29 05:59:59.861000+00:00,2022-11-29 06:00:01.729000+00:00,0000-0003,0,Sell,True,Close,16201.60,0.01,162.0160,16477.05,...,16477.05,0.01,164.7705,2.7545,0.164771,2.589729,-0.01,Doten Short,1.497288,0.00
2022-11-29 05:59:59.861000+00:00,2022-11-29 06:00:01.729000+00:00,0000-0003,1,Sell,True,Open,16477.05,0.01,164.7705,0.00,...,16477.05,0.01,164.7705,0.0000,0.164771,-0.164771,-0.01,Doten Short,1.332518,-0.01
2022-11-30 17:59:59.990000+00:00,2022-11-30 18:00:00.901000+00:00,0000-0004,0,Buy,True,Close,16477.05,0.01,164.7705,16789.68,...,16789.68,0.01,167.8968,-3.1263,0.167897,-3.294197,0.01,doten Long,-1.961679,0.00
2022-11-30 17:59:59.990000+00:00,2022-11-30 18:00:00.901000+00:00,0000-0004,1,Buy,True,Open,16789.68,0.01,167.8968,0.00,...,16789.68,0.01,167.8968,0.0000,0.167897,-0.167897,0.01,doten Long,-2.129576,0.01
2022-11-30 19:59:59.998000+00:00,2022-11-30 20:00:02.857000+00:00,0000-0005,0,Sell,True,Close,16789.68,0.01,167.8968,17061.00,...,17061.00,0.01,170.6100,2.7132,0.170610,2.542590,-0.01,Doten Short,0.413014,0.00
2022-11-30 19:59:59.998000+00:00,2022-11-30 20:00:02.857000+00:00,0000-0005,1,Sell,True,Open,17061.00,0.01,170.6100,0.00,...,17061.00,0.01,170.6100,0.0000,0.170610,-0.170610,-0.01,Doten Short,0.242404,-0.01


In [13]:
df = result_to_df(r)

NameError: name 'result_to_df' is not defined

In [14]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_home_size,open_foreign_size,close_price,...,order_price,order_home_size,order_foreign_size,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-10-05 17:59:59.896000+00:00,2022-10-05 18:00:00.653000+00:00,0000-0001,0,Sell,True,Open,20168.24,0.01,201.6824,0.00,...,20168.24,0.01,201.6824,0.0000,0.020168,-0.020168,-0.01,Open Short,-0.020168,-0.01
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,0,Buy,True,Close,20168.24,0.01,201.6824,19605.10,...,19605.10,0.01,196.0510,5.6314,0.019605,5.611795,0.01,doten Long,5.591627,0.00
2022-10-07 13:59:59.995000+00:00,2022-10-07 14:00:01.114000+00:00,0000-0002,1,Buy,True,Open,19605.10,0.01,196.0510,0.00,...,19605.10,0.01,196.0510,0.0000,0.019605,-0.019605,0.01,doten Long,5.572022,0.01
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,0,Sell,True,Close,19605.10,0.01,196.0510,19474.00,...,19474.00,0.01,194.7400,-1.3110,0.019474,-1.330474,-0.01,Doten Short,4.241548,0.00
2022-10-09 11:59:59.784000+00:00,2022-10-09 12:00:07.204000+00:00,0000-0003,1,Sell,True,Open,19474.00,0.01,194.7400,0.00,...,19474.00,0.01,194.7400,0.0000,0.019474,-0.019474,-0.01,Doten Short,4.222074,-0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0050,1,Buy,True,Open,16949.21,0.01,169.4921,0.00,...,16949.21,0.01,169.4921,0.0000,0.016949,-0.016949,0.01,doten Long,-45.023869,0.01
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,0,Sell,True,Close,16949.21,0.01,169.4921,16968.66,...,16968.66,0.01,169.6866,0.1945,0.016969,0.177531,-0.01,Doten Short,-44.846338,0.00
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0051,1,Sell,True,Open,16968.66,0.01,169.6866,0.00,...,16968.66,0.01,169.6866,0.0000,0.016969,-0.016969,-0.01,Doten Short,-44.863306,-0.01


In [ ]:
df['total_profit'].sum()

In [ ]:
r


In [ ]:
df = result_to_df(r)
df

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time